## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()


In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.2.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jitendra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2492, 4356, 87, 4281, 1461, 4922, 3853, 1416, 4192, 1863],
 [2796, 2412, 3121, 1540, 506, 1442, 1370],
 [2312, 665, 3400, 4831],
 [4708, 3649, 2466, 3716, 1278, 1567],
 [1315, 506, 1428, 4626, 2237, 3829, 506, 1065, 3517, 968],
 [1655,
  1952,
  86,
  1541,
  1172,
  1673,
  225,
  3114,
  3547,
  774,
  211,
  3588,
  3958,
  3311,
  1370],
 [1269, 858, 3399, 3784, 1245, 3825, 4398, 2704, 1060, 2412, 2812],
 [2551, 4737, 4561, 640, 3226, 2598, 1673, 1706, 1060, 2412, 2812],
 [95, 2597, 4975, 2269, 1735, 3451, 4687, 1409, 1673, 381],
 [2686, 4726, 2181, 4533, 1976, 1909, 2481, 3524],
 [4066, 3006, 3621, 3216, 2997, 4257, 4592, 2119, 1646, 1079, 178],
 [3716, 4309, 1461, 3451, 1673, 3226],
 [3558, 4117, 4329, 682, 1582, 3766, 2451, 4744, 2256],
 [3340, 1051, 4511, 3874, 1439, 584, 3101, 1060, 2412, 2812],
 [127, 4145, 691, 2847, 3642, 1060, 2412, 2812],
 [167, 3398, 642, 2642, 52, 2266, 4837, 2225, 521, 2551],
 [591, 4003, 2412],
 [1072, 4407, 1363, 157, 1673, 1004, 4173, 1370],
 [492

### Embedding Representation

In [26]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1416 4192 1863]
 [   0    0    0 ...  506 1442 1370]
 [   0    0    0 ...  665 3400 4831]
 ...
 [   0    0    0 ... 1060 2412 2812]
 [   0    0    0 ...  859   82 1688]
 [   0    0    0 ...  190 3494 3403]]


In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2492,
       4356,   87, 4281, 1461, 4922, 3853, 1416, 4192, 1863], dtype=int32)

In [23]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(18285, (18285,))

In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [28]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 8ms/step - loss: 0.3303 - accuracy: 0.8442 - val_loss: 0.1963 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 1s 6ms/step - loss: 0.1367 - accuracy: 0.9478 - val_loss: 0.2043 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0974 - accuracy: 0.9626 - val_loss: 0.2147 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0727 - accuracy: 0.9752 - val_loss: 0.2472 - val_accuracy: 0.9140
Epoch 5/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0515 - accuracy: 0.9821 - val_loss: 0.2770 - val_accuracy: 0.9097
Epoch 6/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0355 - accuracy: 0.9887 - val_loss: 0.3728 - val_accuracy: 0.9142
Epoch 7/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0257 - accuracy: 0.9926 - val_loss: 0.3705 - val_accuracy: 0.9123
Epoch 

### Performance Metrics And Accuracy

In [29]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test,y_pred)

array([[3119,  300],
       [ 247, 2369]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9093620546810274

### Adding Dropout 

In [31]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Model Training

In [32]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 8ms/step - loss: 0.3490 - accuracy: 0.8320 - val_loss: 0.2094 - val_accuracy: 0.9069
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1530 - accuracy: 0.9404 - val_loss: 0.1964 - val_accuracy: 0.9191
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1025 - accuracy: 0.9638 - val_loss: 0.2151 - val_accuracy: 0.9201
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0713 - accuracy: 0.9761 - val_loss: 0.2589 - val_accuracy: 0.9206
Epoch 5/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0482 - accuracy: 0.9847 - val_loss: 0.2687 - val_accuracy: 0.9140
Epoch 6/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0374 - accuracy: 0.9870 - val_loss: 0.3772 - val_accuracy: 0.9190
Epoch 7/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0257 - accuracy: 0.9917 - val_loss: 0.3498 - val_accuracy: 0.9138
Epoch 

### Performance Metrics And Accuracy

In [33]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_test,y_pred)

array([[3098,  321],
       [ 195, 2421]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9144987572493787